In [10]:
from string import punctuation
import random
import time
import csv
import numpy as np
import sys
import pickle
from spellchecker import SpellChecker
import re
import wordninja

In [11]:
spell = SpellChecker()

# Functions for processing and calculation

In [12]:
def txt_loader(file, listify=True):
    wrapper = open(file,"r",encoding="utf-8-sig")
    data = wrapper.readlines()
    wrapper.close()
    if listify:
        data = [line.translate(str.maketrans('', '', punctuation)).strip().split(' ') for line in data]
    else:
         data = [line.translate(str.maketrans('', '', punctuation)).strip() for line in data]
    
    return data

In [13]:
def pickleLoad(filename):
    start=time.time()
    with open(filename, 'rb') as handle:
        data = pickle.load(handle)
    end=time.time()
    print(str(end-start)+': time elapsed(s)')
    return data

In [14]:
# return True if n_gram contains less than n non-stop words, once false, no more items are added
def lenChecker(n_gram,n):
    clean = [tok for tok in n_gram if tok not in stop_words]
    return len(clean)<n
# given an n-gram as a string, returns n
def whatGram(n_gram):
    clean = n_gram.split(' ')
    clean = [tok for tok in clean if tok not in stop_words]
    return len(clean)
def lenChecker2(n_gram,n):
    clean = [tok for tok in n_gram if tok not in stop_words]
    return len(clean)==n
def lenChecker3(n_gram,n):
    clean = [tok for tok in n_gram.split(' ') if tok not in stop_words]
    return len(clean)==n

In [15]:
def build_unigrams(collection):
    start = time.time()
    unigrams = {}
    for title in collection:
        for word in title:
            
            if word in stop_words or len(word)==0:
                continue
            
            if word in unigrams:
                unigrams[word][0]+=1   
            else:
                # freq,n
                unigrams[word]=[1,1]
    end = time.time()
    print(str(end-start)+': time elapsed(s)')
    return unigrams

In [16]:
def build_bigrams(collection):
    start=time.time()
    bigrams = {}
    for title in collection:
        # -1 as last token must be in prior bigram only
        for i in range(len(title)-1):
            temp=[]
            if title[i] in stop_words:
                continue
            else:
                temp.append(title[i])
                j=1
            while i+j < len(title) and lenChecker(temp,2):
                succ = title[i+j]
                temp.append(succ)
                j+=1

            if len(temp)>1:
                last = temp[len(temp)-1]
                if last in stop_words:
                    continue
                else:
                    temp = ' '.join(temp)
                    if temp in bigrams:
                        bigrams[temp][0]+=1
                    else:
                        bigrams[temp]=[1,2]
    end = time.time()
    print(str(end-start)+': time elapsed(s)')
    return bigrams

In [17]:
def build_trigrams(collection):
    start=time.time()
    trigrams = {}
    for title in collection:
        # -2 as last 2 tokens must be in prior trigram only
        for i in range(len(title)-2):
            temp=[]
            if title[i] in stop_words:
                continue
            else:
                temp.append(title[i])
                j=1
            while i+j < len(title) and lenChecker(temp,3):
                succ = title[i+j]
                temp.append(succ)
                j+=1

            if len(temp)>1:
                last = temp[len(temp)-1]
                if last in stop_words:
                    continue
                elif not lenChecker2(temp,3):
                    continue;
                else:
                    temp = ' '.join(temp)
                    if temp in trigrams:
                        trigrams[temp][0]+=1
                    else:
                        trigrams[temp]=[1,3]
    end = time.time()
    print(str(end-start)+': time elapsed(s)')
    return trigrams

In [18]:
# removes stop-words from complete part of query, investigate differences further
def query_splitter(query):
    query = [a for a in query.split(' ') if a]
    # lower tokens
   # query = [a.lower() if a not in lc_blacklist else a for a in query]
    completed = [a for a in query[:-1] if a not in stop_words]
    partial = query[-1:][0]
    return [completed, partial]


def complete(part):
    #start=time.time()
    completions=[]
    for c in list(unigrams.keys()):
        if c.startswith(part):
            completions.append(c)
    #end = time.time()
   # print(str(end-start)+': time elapsed(s)')
    return completions


In [19]:
# returns the total number of documents containing the term
def df(term):
    # only check unigrams
    if phrases[term][1]==1:
        return phrases[term][0]
    else:
        return 0
# returns the inverse domain frequency of a term
# TODO: change small=>titles

# experiment with omitting +1, assuming df>0
def idf(term):
    return np.log(1+len(small)/1+df(term))
# needs to be changed not to use small

def term_completion_prob(term,denom):
    return (df(term)*idf(term))/denom

# n-gram as string
def freq_norm(n_gram):
    return phrases[n_gram][0] / (np.log(averageFreqs[phrases[n_gram][1]]))

# n-gram as string
def term2phrase(n_gram, term):
    #start=time.time()
    if term in n_gram:
        #out = freq_norm(n_gram) / sum([freq_norm(a) for a in t2p[term][0]])
        out = freq_norm(n_gram) / t2p[term][1]
    else:
        out = 0
    #end=time.time()
   # print(end-start)
    return out

# n-gram as string
def phrase_selection_prob(n_gram,term,completions,denom):
    #return term2phrase(n_gram,term) * term_completion_prob(term,completions,denom)
    return term2phrase(n_gram,term) * term_completion_prob(term,completions,denom)

In [20]:
## Build Inverted Index for finding the phrases containing a searched term
# {term:[phrase]} containing term

def t2p_build():
    P = {}
    start = time.time()
    for p in list(phrases.keys()):
        for word in p.split(' '):
            if word in P:
                # p.split(' ')
                P[word].append(p)
            else:
                # p.split(' ')
                P[word] = [p]
    
    # store sum of freq norm for all phrases relating to a term
    # so this may be accessed by term2phrase and not calculated at runtime
    for k in P:
        P[k] = (P[k],sum([freq_norm(a) for a in P[k]]))           
    end = time.time()
    print(str(end-start)+': time elapsed(s)')
    return P

In [21]:
# maps terms to docIDs
def t2ID_build():    
    start=time.time()
    idx = {}
    for c1, d in enumerate(small):
        #tokens = preprocessor(d[0])
        for tok in d:
            if tok in stop_words:
                continue
            if tok in idx:
                # inverted index is dict mapping term to list of docIDs
                idx[tok].add(c1)
            else:
                idx[tok] = {c1}
    end = time.time()
    print(str(end-start)+': time elapsed(s)')
    return idx

In [22]:
# returns the docIDs of any documents containing all terms present in an input sequence
# unions the set of docIDs for each term to produce a set containing each term in the phrase
def getIDs(seq):
    phrase = [tok for tok in seq if tok not in stop_words]
    if phrase:
        try:
            out = t2ID[phrase[0]]
        except KeyError:
            return set()
        phrase=phrase[1:]
        if phrase:
            for i in phrase:
                 out = out.intersection(t2ID[i])
    else:
        return set()
    return out

In [23]:
# n-gram as str
def query_correlation(n_gram, full):
    #start=time.time()
    phrase_IDs = getIDs(n_gram.split(' '))
    full_IDs = getIDs(full)
    if full_IDs:
        check = all([True if word in n_gram else False for word in full])
    if phrase_IDs and check:
        #print(len(phrase_IDs.intersection(full_IDs)))
        #print(len(phrase_IDs))     
        out = len(phrase_IDs.intersection(full_IDs))/len(phrase_IDs)
    else:
        out = 0
    #end=time.time()
    #print(str(end-start)+': time elapsed(s)')
    return out

## Main function

In [24]:
def run(query):
    start=time.time()
    full, partial = query_splitter(query)
    completions = complete(partial)
    # calc ahead of time as denom is constant given query
    denom = sum([df(c)*idf(c) for c in completions])
    # C will hold candidate phrases and their scores
    C=[]
    for c in completions:
        # iterate over possible word completions
        tc_prob = term_completion_prob(c,denom)
        for p in t2p[c][0]: 
            # for each completion, iterate over all phrases containing the completion
            t2p_prob = term2phrase(p,c)
            if full:
                qp_corr = query_correlation(p,full)
                score = tc_prob * t2p_prob * qp_corr
            else:
                 score = tc_prob * t2p_prob
            # add phrase and its score to output
            if score>0:
                C.append((p,score))
    # sort by score
    C.sort(key=lambda tup: tup[1],reverse=True)
    end=time.time()
    print(str(end-start)+': time elapsed(s)')
    return [a[0] for a in C[:8]]

# Create Variables

In [25]:
stop_words = set(txt_loader("englishST.txt",False))

In [26]:
# small is our sub-dataset consisting of 100k titles
small = pickleLoad('100k.pickle')
# remove empty items, occasional titles contain ['']
small = [list(filter(None, str_list)) for str_list in small]
print('Example title format: \n'+str(small[2121]))
k=50000
small = random.sample(small,k)

0.23845911026000977: time elapsed(s)
Example title format: 
['Easy', 'way', 'to', 'add', 'standardUserDefaults', 'to', 'Crittercism', 'error', 'report', 'metadata']


In [27]:
small = pickleLoad('1m.pickle')
k = 80000
small = random.sample(small,k)

2.9920730590820312: time elapsed(s)


# Preprocess Block

###### - SpellCheck terms, identify unknown tokens
###### - build Camel case blacklist
###### - Camel case handling to split CC tokens
###### - Multi-word case handling to split MW tokens : getOrderValue -> get, Order, Value

In [28]:
# camelCase blacklist
cc_pattern = re.compile("[A-Z]*[^A-Z]+")
cc_blacklist = ['jQuery','MySQL','JavaScript']
lc_blacklist = ['I',"I'm","I'll","I'd"]
sc = SpellChecker()

In [29]:
# split camel case tokens
for title in small:
    knowns = sc.known(title)
    # get unknown tokens and their indexes
    unknowns = [tok for tok in title if tok.lower() not in knowns]
    for i in unknowns:
        # check if token is camelCase and can be split
        if cc_pattern.findall(i) and i not in cc_blacklist:
            idx = title.index(i)
            title.remove(i)
            for c2,j in enumerate(cc_pattern.findall(i)):
                title.insert(idx+c2,j)

In [ ]:
# split multi word tokens
for title in small:
    knowns = sc.known(title)
    # get unknown tokens and their indexes
    unknowns = [tok for tok in title if tok.lower() not in knowns]
    for i in unknowns:
        if wordninja.split(i):
            idx = title.index(i)
            title.remove(i)
            for c,tok in enumerate(wordninja.split(i)):
                title.insert(idx+c,tok)

small = [[a.lower() if a not in lc_blacklist else a for a in title] for title in small]

# Build models/structures

In [ ]:
# as we build our n-grams, calc the average frequency for each n, and add into averageFreqs
averageFreqs = {}

In [ ]:
unigrams = build_unigrams(small)
averageFreqs[1] = sum([a[0] for a in list(unigrams.values())])/(len(list(unigrams.keys())))

In [ ]:
bigrams = build_bigrams(small)
averageFreqs[2] = sum([a[0] for a in list(bigrams.values())])/(len(list(bigrams.keys())))

In [ ]:
trigrams=build_trigrams(small)
averageFreqs[3] = sum([a[0] for a in list(trigrams.values())])/(len(list(trigrams.keys())))

In [ ]:
print('unigrams/vocab size: '+str(len(unigrams.keys())))
print('bigrams: '+str(len(bigrams.keys())) )
print( 'trigrams: '+str(len(trigrams.keys())) +'\n')
print('Average freq of n-grams for each order n:\n'+str(averageFreqs))

In [22]:
# phrases maps {n-gram:[freq,n]}
phrases = {}
phrases.update(unigrams)
phrases.update(bigrams)
phrases.update(trigrams)

# bigrams and trigrams no longer needed, can be freed from memory, retain unigrams to serve as vocab
del bigrams, trigrams

In [23]:
# build term to [phrase] index
t2p = t2p_build()

5.802832126617432: time elapsed(s)


In [24]:
# build term to [docID] index
t2ID = t2ID_build()

0.3256101608276367: time elapsed(s)


# Run / Test

In [34]:
run("postgres l")

0.5543758869171143: time elapsed(s)


['postgres sql language',
 'variable value to postgres ltree',
 'postgres ltree column',
 'log using triggers in postgres',
 'TCPNODELAY for libpq and postgres',
 'libpq and postgres server',
 'large file in postgres',
 'postgres ltree']

In [35]:
run("secure s")

1.277764081954956: time elapsed(s)


['secure than JavaScript solutions',
 'secure HTTP server',
 'secure socket server',
 'reactJs secure storage',
 'secure my data in sdcard',
 'method for secure socket',
 'secure socket server',
 'Pass session through secure']

In [64]:
run("sql m")

0.6444478034973145: time elapsed(s)


['sql server management',
 'microsoft sql server',
 'ms sql server',
 'ms sql',
 'sql server mode',
 'ms sql query',
 'ms sql database',
 'php ms sql']

# Notes